In [1]:
import pandas as pd
import numpy as np

clients_raw = pd.read_csv("DATA/clients_202510280926.csv")
orders_raw = pd.read_csv("DATA/orders_202510280929.csv")
products_raw = pd.read_csv("DATA/products_202510280931.csv")

clients_raw.head(), orders_raw.head(), products_raw.head()


(                              client_id      first_name last_name gender  age  \
 0  21e5c13d-1c9a-4d00-9164-b72302d5edef         Tristán     Rojas      M   42   
 1  36e48bdd-db11-4abe-9526-cfc90e68924d           Óscar  Barranco      M   58   
 2  145c22df-3579-412e-bc12-b4fce70abaf3         Macario    Querol      M   28   
 3  90c4a925-e51f-4dac-9193-2d9aec97a472          Ramiro   Sanchez      M   19   
 4  853f711a-4c36-40b4-b5d0-6207152cd793  Juan Francisco      Flor      M   36   
 
                                email      city               address  \
 0        tristán.rojas13@example.com    Rivera  Bulevar Artigas 3757   
 1       óscar.barranco13@example.com     Salto  Bulevar Artigas 3911   
 2       macario.querol67@example.com    Rivera        Av. Rivera 206   
 3       ramiro.sanchez53@example.com     Salto  Av. 18 de Julio 6324   
 4  juan francisco.flor68@example.com  Mercedes  Av. 18 de Julio 6301   
 
    postal_code  country citizenship document_type document_number

In [2]:
# --- LIMPIEZA DE CLIENTS ---

clients = clients_raw.copy()

# Forzar tipos string
str_cols = [
    "client_id", "first_name", "last_name", "gender",
    "email", "city", "address", "postal_code",
    "country", "citizenship", "document_type",
    "document_number"
]

for col in str_cols:
    clients[col] = clients[col].astype(str).str.strip()

# Convertir edad a numérico
clients["age"] = pd.to_numeric(clients["age"], errors="coerce")

# Convertir fechas
clients["registration_date"] = pd.to_datetime(clients["registration_date"], errors="coerce")
clients["last_seen"] = pd.to_datetime(clients["last_seen"], errors="coerce")

# Duplicados
dup_count = clients.duplicated().sum()
print(f"Duplicados en clients: {dup_count}")
clients = clients.drop_duplicates()

# Nulos por columna
print("\nNulos por columna:")
print(clients.isnull().sum())

clients.head()


Duplicados en clients: 0

Nulos por columna:
client_id            0
first_name           0
last_name            0
gender               0
age                  0
email                0
city                 0
address              0
postal_code          0
country              0
citizenship          0
document_type        0
document_number      0
registration_date    0
last_seen            0
dtype: int64


,client_id,first_name,last_name,gender,age,email,city,address,postal_code,country,citizenship,document_type,document_number,registration_date,last_seen
0,21e5c13d-1c9a-4d00-9164-b72302d5edef,Tristán,Rojas,M,42,tristán.rojas13@example.com,Rivera,Bulevar Artigas 3757,28289,Uruguay,Uruguay,CI,2719583-8,2025-05-02,2025-06-19
1,36e48bdd-db11-4abe-9526-cfc90e68924d,Óscar,Barranco,M,58,óscar.barranco13@example.com,Salto,Bulevar Artigas 3911,76237,Uruguay,Uruguay,CI,1445199-8,2023-02-24,2023-12-09
2,145c22df-3579-412e-bc12-b4fce70abaf3,Macario,Querol,M,28,macario.querol67@example.com,Rivera,Av. Rivera 206,30926,Uruguay,Uruguay,CI,8090293-5,2024-10-03,2025-05-03
3,90c4a925-e51f-4dac-9193-2d9aec97a472,Ramiro,Sanchez,M,19,ramiro.sanchez53@example.com,Salto,Av. 18 de Julio 6324,22676,Uruguay,Uruguay,CI,7022674-5,2025-01-13,2025-08-11
4,853f711a-4c36-40b4-b5d0-6207152cd793,Juan Francisco,Flor,M,36,juan francisco.flor68@example.com,Mercedes,Av. 18 de Julio 6301,20328,Uruguay,Uruguay,CI,5918715-9,2025-03-09,2025-03-21


In [3]:
orders_raw.head()


,order_id,client_id,product_id,product_category,order_price,order_timestamp,order_status,payment_method,shipping_method,shipping_cost
0,1,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00333,Snacks,29.51,2023-01-01 23:51:55.000,delivered,bank_transfer,standard,3.51
1,2,0c8380cb-a64e-4090-8b23-d1ddf32630e0,P00929,Personal Care,23.40,2023-01-01 11:48:50.000,delivered,credit_card,standard,3.76
2,3,bf486527-8182-490e-9e10-c79232c6b0c9,P00768,Personal Care,94.41,2023-01-01 11:06:14.000,delivered,debit_card,express,10.42
3,5,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00610,Household,29.16,2023-01-01 20:55:35.000,delivered,credit_card,standard,6.05
4,6,7ac7a850-4f19-4c85-921b-b0e5ba921ee9,P00299,Snacks,76.22,2023-01-01 02:49:09.000,delivered,debit_card,express,14.08


In [4]:
# --- LIMPIEZA DE ORDERS ---

orders = orders_raw.copy()

# Forzar columnas a string
str_cols = [
    "order_id", "client_id", "product_id", "product_category",
    "order_status", "payment_method", "shipping_method"
]

for col in str_cols:
    orders[col] = orders[col].astype(str).str.strip()

# Convertir a numéricos
orders["order_price"] = pd.to_numeric(orders["order_price"], errors="coerce")
orders["shipping_cost"] = pd.to_numeric(orders["shipping_cost"], errors="coerce")

# Convertir a fecha
orders["order_timestamp"] = pd.to_datetime(orders["order_timestamp"], errors="coerce")

# Buscar duplicados
dup_orders = orders.duplicated().sum()
print("Duplicados en orders:", dup_orders)

# Eliminar duplicados si existieran
orders = orders.drop_duplicates()

# Nulos
print("\nNulos por columna:")
print(orders.isnull().sum())

# Mostrar head para validar
orders.head()


Duplicados en orders: 0

Nulos por columna:
order_id            0
client_id           0
product_id          0
product_category    0
order_price         0
order_timestamp     0
order_status        0
payment_method      0
shipping_method     0
shipping_cost       0
dtype: int64


,order_id,client_id,product_id,product_category,order_price,order_timestamp,order_status,payment_method,shipping_method,shipping_cost
0,1,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00333,Snacks,29.51,2023-01-01 23:51:55,delivered,bank_transfer,standard,3.51
1,2,0c8380cb-a64e-4090-8b23-d1ddf32630e0,P00929,Personal Care,23.40,2023-01-01 11:48:50,delivered,credit_card,standard,3.76
2,3,bf486527-8182-490e-9e10-c79232c6b0c9,P00768,Personal Care,94.41,2023-01-01 11:06:14,delivered,debit_card,express,10.42
3,5,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00610,Household,29.16,2023-01-01 20:55:35,delivered,credit_card,standard,6.05
4,6,7ac7a850-4f19-4c85-921b-b0e5ba921ee9,P00299,Snacks,76.22,2023-01-01 02:49:09,delivered,debit_card,express,14.08


In [5]:
products_raw.head()


,product_id,product_name,category,brand,last_updated,created_at
0,P00001,Lipton Ice Tea,Beverages,Unilever,2025-09-06 21:21:31.000,2025-04-06 21:21:31.000
1,P00002,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31.000,2025-05-02 21:21:31.000
2,P00003,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31.000,2024-10-02 21:21:31.000
3,P00004,Lipton Ice Tea,Beverages,Unilever,2025-09-06 21:21:31.000,2025-04-21 21:21:31.000
4,P00005,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31.000,2025-04-22 21:21:31.000


In [6]:
# --- LIMPIEZA DE PRODUCTS ---

products = products_raw.copy()

# Convertir columnas a string
str_cols = ["product_id", "product_name", "category", "brand"]

for col in str_cols:
    products[col] = products[col].astype(str).str.strip()

# Convertir fechas
products["last_updated"] = pd.to_datetime(products["last_updated"], errors="coerce")
products["created_at"] = pd.to_datetime(products["created_at"], errors="coerce")

# Buscar duplicados
dup_products = products.duplicated().sum()
print("Duplicados en products:", dup_products)

# Eliminar duplicados
products = products.drop_duplicates()

# Nulos
print("\nNulos por columna:")
print(products.isnull().sum())

# Mostrar head final
products.head()


Duplicados en products: 0

Nulos por columna:
product_id      0
product_name    0
category        0
brand           0
last_updated    0
created_at      0
dtype: int64


,product_id,product_name,category,brand,last_updated,created_at
0,P00001,Lipton Ice Tea,Beverages,Unilever,2025-09-06 21:21:31,2025-04-06 21:21:31
1,P00002,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31,2025-05-02 21:21:31
2,P00003,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31,2024-10-02 21:21:31
3,P00004,Lipton Ice Tea,Beverages,Unilever,2025-09-06 21:21:31,2025-04-21 21:21:31
4,P00005,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31,2025-04-22 21:21:31


In [7]:
snapshot_date = orders["order_timestamp"].max()
snapshot_date


Timestamp('2024-12-31 23:47:40')

In [8]:
# Última compra de cada cliente
last_purchase = (
    orders.groupby("client_id")["order_timestamp"]
    .max()
    .reset_index()
    .rename(columns={"order_timestamp": "last_purchase_date"})
)

last_purchase.head()


,client_id,last_purchase_date
0,00094d4c-1acd-46ec-a6d2-c35710c31005,2024-04-27 17:58:43
1,0016d161-d6a2-42f3-bfd6-66aee6d163e3,2023-07-11 15:17:43
2,0042e01f-9154-45fd-9c1d-499b3eb3f1c4,2024-12-28 16:17:46
3,0052b716-46cf-4540-aa23-409642dc4048,2024-12-28 12:43:15
4,00679c7c-bd80-4a70-9052-830bc5b63ecf,2024-12-17 22:38:56


In [9]:
# Cálculo de días desde última compra
last_purchase["days_since_last_purchase"] = (
    snapshot_date - last_purchase["last_purchase_date"]
).dt.days

last_purchase.head()


,client_id,last_purchase_date,days_since_last_purchase
0,00094d4c-1acd-46ec-a6d2-c35710c31005,2024-04-27 17:58:43,248
1,0016d161-d6a2-42f3-bfd6-66aee6d163e3,2023-07-11 15:17:43,539
2,0042e01f-9154-45fd-9c1d-499b3eb3f1c4,2024-12-28 16:17:46,3
3,0052b716-46cf-4540-aa23-409642dc4048,2024-12-28 12:43:15,3
4,00679c7c-bd80-4a70-9052-830bc5b63ecf,2024-12-17 22:38:56,14


In [10]:
# Definir churn con umbral de 90 días
CHURN_THRESHOLD = 90

last_purchase["churn"] = (last_purchase["days_since_last_purchase"] > CHURN_THRESHOLD).astype(int)

last_purchase.head()


,client_id,last_purchase_date,days_since_last_purchase,churn
0,00094d4c-1acd-46ec-a6d2-c35710c31005,2024-04-27 17:58:43,248,1
1,0016d161-d6a2-42f3-bfd6-66aee6d163e3,2023-07-11 15:17:43,539,1
2,0042e01f-9154-45fd-9c1d-499b3eb3f1c4,2024-12-28 16:17:46,3,0
3,0052b716-46cf-4540-aa23-409642dc4048,2024-12-28 12:43:15,3,0
4,00679c7c-bd80-4a70-9052-830bc5b63ecf,2024-12-17 22:38:56,14,0


In [11]:
# Dataset analítico base del cliente
client_base = clients.merge(
    last_purchase[["client_id", "last_purchase_date", "days_since_last_purchase", "churn"]],
    on="client_id",
    how="left"
)

client_base.head()


,client_id,first_name,last_name,gender,age,email,city,address,postal_code,country,citizenship,document_type,document_number,registration_date,last_seen,last_purchase_date,days_since_last_purchase,churn
0,21e5c13d-1c9a-4d00-9164-b72302d5edef,Tristán,Rojas,M,42,tristán.rojas13@example.com,Rivera,Bulevar Artigas 3757,28289,Uruguay,Uruguay,CI,2719583-8,2025-05-02,2025-06-19,2024-12-16 21:01:32,15.0,0.0
1,36e48bdd-db11-4abe-9526-cfc90e68924d,Óscar,Barranco,M,58,óscar.barranco13@example.com,Salto,Bulevar Artigas 3911,76237,Uruguay,Uruguay,CI,1445199-8,2023-02-24,2023-12-09,2024-12-26 10:58:03,5.0,0.0
2,145c22df-3579-412e-bc12-b4fce70abaf3,Macario,Querol,M,28,macario.querol67@example.com,Rivera,Av. Rivera 206,30926,Uruguay,Uruguay,CI,8090293-5,2024-10-03,2025-05-03,2024-12-29 22:46:06,2.0,0.0
3,90c4a925-e51f-4dac-9193-2d9aec97a472,Ramiro,Sanchez,M,19,ramiro.sanchez53@example.com,Salto,Av. 18 de Julio 6324,22676,Uruguay,Uruguay,CI,7022674-5,2025-01-13,2025-08-11,2024-11-19 02:28:56,42.0,0.0
4,853f711a-4c36-40b4-b5d0-6207152cd793,Juan Francisco,Flor,M,36,juan francisco.flor68@example.com,Mercedes,Av. 18 de Julio 6301,20328,Uruguay,Uruguay,CI,5918715-9,2025-03-09,2025-03-21,NaT,NaN,NaN


In [12]:
# Completar clientes sin compras (NaT) como churn = 1
client_base["churn"] = client_base["churn"].fillna(1).astype(int)

# Para days_since_last_purchase asignamos un valor muy alto (por ejemplo 999)
client_base["days_since_last_purchase"] = client_base["days_since_last_purchase"].fillna(999)

client_base.head()



,client_id,first_name,last_name,gender,age,email,city,address,postal_code,country,citizenship,document_type,document_number,registration_date,last_seen,last_purchase_date,days_since_last_purchase,churn
0,21e5c13d-1c9a-4d00-9164-b72302d5edef,Tristán,Rojas,M,42,tristán.rojas13@example.com,Rivera,Bulevar Artigas 3757,28289,Uruguay,Uruguay,CI,2719583-8,2025-05-02,2025-06-19,2024-12-16 21:01:32,15.0,0
1,36e48bdd-db11-4abe-9526-cfc90e68924d,Óscar,Barranco,M,58,óscar.barranco13@example.com,Salto,Bulevar Artigas 3911,76237,Uruguay,Uruguay,CI,1445199-8,2023-02-24,2023-12-09,2024-12-26 10:58:03,5.0,0
2,145c22df-3579-412e-bc12-b4fce70abaf3,Macario,Querol,M,28,macario.querol67@example.com,Rivera,Av. Rivera 206,30926,Uruguay,Uruguay,CI,8090293-5,2024-10-03,2025-05-03,2024-12-29 22:46:06,2.0,0
3,90c4a925-e51f-4dac-9193-2d9aec97a472,Ramiro,Sanchez,M,19,ramiro.sanchez53@example.com,Salto,Av. 18 de Julio 6324,22676,Uruguay,Uruguay,CI,7022674-5,2025-01-13,2025-08-11,2024-11-19 02:28:56,42.0,0
4,853f711a-4c36-40b4-b5d0-6207152cd793,Juan Francisco,Flor,M,36,juan francisco.flor68@example.com,Mercedes,Av. 18 de Julio 6301,20328,Uruguay,Uruguay,CI,5918715-9,2025-03-09,2025-03-21,NaT,999.0,1


In [14]:
# Frequency: número de órdenes por cliente
frequency = (
    orders.groupby("client_id")["order_id"]
    .count()
    .reset_index()
    .rename(columns={"order_id": "frequency"})
)

frequency.head()


,client_id,frequency
0,00094d4c-1acd-46ec-a6d2-c35710c31005,2
1,0016d161-d6a2-42f3-bfd6-66aee6d163e3,4
2,0042e01f-9154-45fd-9c1d-499b3eb3f1c4,77
3,0052b716-46cf-4540-aa23-409642dc4048,67
4,00679c7c-bd80-4a70-9052-830bc5b63ecf,79


In [15]:
# Monetary: gasto total por cliente
monetary = (
    orders.groupby("client_id")["order_price"]
    .sum()
    .reset_index()
    .rename(columns={"order_price": "monetary"})
)

monetary.head()

,client_id,monetary
0,00094d4c-1acd-46ec-a6d2-c35710c31005,133.85
1,0016d161-d6a2-42f3-bfd6-66aee6d163e3,235.11
2,0042e01f-9154-45fd-9c1d-499b3eb3f1c4,3943.76
3,0052b716-46cf-4540-aa23-409642dc4048,3590.10
4,00679c7c-bd80-4a70-9052-830bc5b63ecf,4489.99


In [16]:
rfm = frequency.merge(monetary, on="client_id", how="outer")
rfm.head()


,client_id,frequency,monetary
0,00094d4c-1acd-46ec-a6d2-c35710c31005,2,133.85
1,0016d161-d6a2-42f3-bfd6-66aee6d163e3,4,235.11
2,0042e01f-9154-45fd-9c1d-499b3eb3f1c4,77,3943.76
3,0052b716-46cf-4540-aa23-409642dc4048,67,3590.10
4,00679c7c-bd80-4a70-9052-830bc5b63ecf,79,4489.99


In [17]:
# Unir RFM con dataset base de clientes
client_full = client_base.merge(rfm, on="client_id", how="left")

client_full.head()


,client_id,first_name,last_name,gender,age,email,city,address,postal_code,country,citizenship,document_type,document_number,registration_date,last_seen,last_purchase_date,days_since_last_purchase,churn,frequency,monetary
0,21e5c13d-1c9a-4d00-9164-b72302d5edef,Tristán,Rojas,M,42,tristán.rojas13@example.com,Rivera,Bulevar Artigas 3757,28289,Uruguay,Uruguay,CI,2719583-8,2025-05-02,2025-06-19,2024-12-16 21:01:32,15.0,0,44.0,2306.18
1,36e48bdd-db11-4abe-9526-cfc90e68924d,Óscar,Barranco,M,58,óscar.barranco13@example.com,Salto,Bulevar Artigas 3911,76237,Uruguay,Uruguay,CI,1445199-8,2023-02-24,2023-12-09,2024-12-26 10:58:03,5.0,0,122.0,7236.93
2,145c22df-3579-412e-bc12-b4fce70abaf3,Macario,Querol,M,28,macario.querol67@example.com,Rivera,Av. Rivera 206,30926,Uruguay,Uruguay,CI,8090293-5,2024-10-03,2025-05-03,2024-12-29 22:46:06,2.0,0,37.0,2076.58
3,90c4a925-e51f-4dac-9193-2d9aec97a472,Ramiro,Sanchez,M,19,ramiro.sanchez53@example.com,Salto,Av. 18 de Julio 6324,22676,Uruguay,Uruguay,CI,7022674-5,2025-01-13,2025-08-11,2024-11-19 02:28:56,42.0,0,13.0,614.39
4,853f711a-4c36-40b4-b5d0-6207152cd793,Juan Francisco,Flor,M,36,juan francisco.flor68@example.com,Mercedes,Av. 18 de Julio 6301,20328,Uruguay,Uruguay,CI,5918715-9,2025-03-09,2025-03-21,NaT,999.0,1,NaN,NaN


In [18]:
# Completar features RFM para clientes sin compras
client_full["frequency"] = client_full["frequency"].fillna(0).astype(int)
client_full["monetary"] = client_full["monetary"].fillna(0)

client_full.head()


,client_id,first_name,last_name,gender,age,email,city,address,postal_code,country,citizenship,document_type,document_number,registration_date,last_seen,last_purchase_date,days_since_last_purchase,churn,frequency,monetary
0,21e5c13d-1c9a-4d00-9164-b72302d5edef,Tristán,Rojas,M,42,tristán.rojas13@example.com,Rivera,Bulevar Artigas 3757,28289,Uruguay,Uruguay,CI,2719583-8,2025-05-02,2025-06-19,2024-12-16 21:01:32,15.0,0,44,2306.18
1,36e48bdd-db11-4abe-9526-cfc90e68924d,Óscar,Barranco,M,58,óscar.barranco13@example.com,Salto,Bulevar Artigas 3911,76237,Uruguay,Uruguay,CI,1445199-8,2023-02-24,2023-12-09,2024-12-26 10:58:03,5.0,0,122,7236.93
2,145c22df-3579-412e-bc12-b4fce70abaf3,Macario,Querol,M,28,macario.querol67@example.com,Rivera,Av. Rivera 206,30926,Uruguay,Uruguay,CI,8090293-5,2024-10-03,2025-05-03,2024-12-29 22:46:06,2.0,0,37,2076.58
3,90c4a925-e51f-4dac-9193-2d9aec97a472,Ramiro,Sanchez,M,19,ramiro.sanchez53@example.com,Salto,Av. 18 de Julio 6324,22676,Uruguay,Uruguay,CI,7022674-5,2025-01-13,2025-08-11,2024-11-19 02:28:56,42.0,0,13,614.39
4,853f711a-4c36-40b4-b5d0-6207152cd793,Juan Francisco,Flor,M,36,juan francisco.flor68@example.com,Mercedes,Av. 18 de Julio 6301,20328,Uruguay,Uruguay,CI,5918715-9,2025-03-09,2025-03-21,NaT,999.0,1,0,0.00


In [19]:
# Guardar dataset analítico final
client_full.to_csv("DATA/client_full.csv", index=False)

client_full.shape


(2357, 20)